In [1]:
#Base Libraries
import os 
import sys
import json
import csv

#Core Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pygwalker as pyg
import datetime as dt
#Model Libraries
from bertopic import BERTopic
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
from sentence_transformers import SentenceTransformer

#ML Libraries
import torch 


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/V

In [2]:

#Importing Configs
# Define the path where config.py is located
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local

/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling
/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling
/Users/kylenabors/Documents


In [3]:
df = pd.read_csv("/Users/kylenabors/Documents/MS-Thesis Data/Database/Fed Data/fed_data_blocks.csv")
docs = df["segment"].to_list()
timestamps = df['date'].to_list()
type = df['type'].to_list()

#BERT Models
topic_model_BB = torch.load(f"{bert_models_local}/topic_model_fed_BB.pt")
topic_model_all = torch.load(f"{bert_models_local}/topic_model_fed_all.pt")

Wed Sep 13 15:44:20 2023 Building and compiling search function


In [4]:
topics_over_time = topic_model_BB.topics_over_time(docs, timestamps, nr_bins=100)

df_tot = pd.DataFrame(topics_over_time, columns=['Topic', 'Words', 'Frequency', 'Timestamp'])
df_tot['Timestamp'] = pd.to_datetime(df_tot['Timestamp'], format='%Y-%m-%d').dt.to_period('D')
df_tot.to_csv(f"{bert_models_local}/tot.csv", index=True)

100it [01:00,  1.66it/s]


In [5]:

      
pivot_df_tot = df_tot.pivot(index='Timestamp', columns='Topic', values='Frequency')

pivot_df_tot = pivot_df_tot.fillna(0)
print(pivot_df_tot.head())

pivot_df_tot.to_csv(f"{bert_models_local}/pivot_df_tot.csv", index=True)


Topic        -1     0    1    2    3    4    5    6    7    8   ...   21   22  \
Timestamp                                                       ...             
2007-03-01  21.0  22.0  0.0  8.0  0.0  2.0  2.0  1.0  0.0  1.0  ...  0.0  0.0   
2007-05-05   6.0  14.0  0.0  2.0  0.0  3.0  1.0  2.0  0.0  1.0  ...  0.0  0.0   
2007-07-04  11.0   9.0  1.0  1.0  2.0  1.0  2.0  1.0  0.0  0.0  ...  0.0  0.0   
2007-09-02  21.0  18.0  1.0  4.0  3.0  2.0  4.0  3.0  0.0  0.0  ...  0.0  0.0   
2007-10-31   2.0   0.0  0.0  2.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0   

Topic        23   24   25   26   27   28   29   30  
Timestamp                                           
2007-03-01  0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  
2007-05-05  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
2007-07-04  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
2007-09-02  0.0  0.0  0.0  1.0  0.0  2.0  0.0  0.0  
2007-10-31  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 32 columns]


In [6]:
gwalker = pyg.walk(df_tot)

Box(children=(HTML(value='<div id="ifr-pyg-0" style="height: auto">\n    <head>\n        <meta http-equiv="Con…